In [134]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVC,LinearSVC

In [94]:
train=pd.read_csv('Train.csv')
test=pd.read_csv('test.csv')

In [95]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,Male,No,0,Graduate,No,5849,0.0,NaN,360.0,1.0,Urban,Y
1,LP001003,Male,Yes,1,Graduate,No,4583,1508.0,128.0,360.0,1.0,Rural,N
2,LP001005,Male,Yes,0,Graduate,Yes,3000,0.0,66.0,360.0,1.0,Urban,Y
3,LP001006,Male,Yes,0,Not Graduate,No,2583,2358.0,120.0,360.0,1.0,Urban,Y
4,LP001008,Male,No,0,Graduate,No,6000,0.0,141.0,360.0,1.0,Urban,Y


In [96]:
test.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area
0,LP001015,Male,Yes,0,Graduate,No,5720,0,110.0,360.0,1.0,Urban
1,LP001022,Male,Yes,1,Graduate,No,3076,1500,126.0,360.0,1.0,Urban
2,LP001031,Male,Yes,2,Graduate,No,5000,1800,208.0,360.0,1.0,Urban
3,LP001035,Male,Yes,2,Graduate,No,2340,2546,100.0,360.0,NaN,Urban
4,LP001051,Male,No,0,Not Graduate,No,3276,0,78.0,360.0,1.0,Urban


In [97]:
print(train.shape,' ' ,test.shape)

(614, 13)   (367, 12)


In [98]:
train.nunique()

Loan_ID              614
Gender                 2
Married                2
Dependents             4
Education              2
Self_Employed          2
ApplicantIncome      505
CoapplicantIncome    287
LoanAmount           203
Loan_Amount_Term      10
Credit_History         2
Property_Area          3
Loan_Status            2
dtype: int64

In [99]:
train.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [100]:
cat=['Gender','Married','Education','Self_Employed','Property_Area','Loan_Status']
le=LabelEncoder()

In [101]:
replace_dict = {
            "Married": {"Yes": 1, "No": 0},
            "Education": {"Graduate": 1, "Not Graduate": 0},
            "Gender": {"Male": 1, "Female": 0},
            "Property_Area": {"Urban": 2, "Semiurban": 1, "Rural": 0},
            "Self_Employed": {"Yes": 1, "No": 0},
            "Loan_Status": {"Y": 1, "N": 0},
            "Dependents": {"3+": 4}
}
train = train.replace(replace_dict)

In [102]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,LP001002,1.0,0.0,0,1,0.0,5849,0.0,NaN,360.0,1.0,2,1
1,LP001003,1.0,1.0,1,1,0.0,4583,1508.0,128.0,360.0,1.0,0,0
2,LP001005,1.0,1.0,0,1,1.0,3000,0.0,66.0,360.0,1.0,2,1
3,LP001006,1.0,1.0,0,0,0.0,2583,2358.0,120.0,360.0,1.0,2,1
4,LP001008,1.0,0.0,0,1,0.0,6000,0.0,141.0,360.0,1.0,2,1


In [103]:
train.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

In [104]:
train['loan_null']=train['Gender'].isnull()
train['married_null']=train['Married'].isnull()
train['dependents_null']=train['Dependents'].isnull()
train['self_employed_null']=train['Self_Employed'].isnull()
train['Loan_Amount_null']=train['LoanAmount'].isnull()
train['term_null']=train['Loan_Amount_Term'].isnull()
train['History_null']=train['Credit_History'].isnull()

In [105]:
test['loan_null']=test['Gender'].isnull()
test['married_null']=test['Married'].isnull()
test['dependents_null']=test['Dependents'].isnull()
test['self_employed_null']=test['Self_Employed'].isnull()
test['Loan_Amount_null']=test['LoanAmount'].isnull()
test['term_null']=test['Loan_Amount_Term'].isnull()
test['History_null']=test['Credit_History'].isnull()

In [106]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,loan_null,married_null,dependents_null,self_employed_null,Loan_Amount_null,term_null,History_null
0,LP001002,1.0,0.0,0,1,0.0,5849,0.0,NaN,360.0,1.0,2,1,False,False,False,False,True,False,False
1,LP001003,1.0,1.0,1,1,0.0,4583,1508.0,128.0,360.0,1.0,0,0,False,False,False,False,False,False,False
2,LP001005,1.0,1.0,0,1,1.0,3000,0.0,66.0,360.0,1.0,2,1,False,False,False,False,False,False,False
3,LP001006,1.0,1.0,0,0,0.0,2583,2358.0,120.0,360.0,1.0,2,1,False,False,False,False,False,False,False
4,LP001008,1.0,0.0,0,1,0.0,6000,0.0,141.0,360.0,1.0,2,1,False,False,False,False,False,False,False


In [107]:
train.loc[:, 'Married'] = train.loc[:, 'Married'].fillna(0)
train.loc[:, 'Gender'] = train.loc[:, 'Gender'].fillna(0)
train.loc[:, 'Dependents'] = train.loc[:, 'Dependents'].fillna(0)
train.loc[:, 'Self_Employed'] = train.loc[:, 'Self_Employed'].fillna(0)
train.loc[:, 'LoanAmount'] = train.loc[:, 'LoanAmount'].fillna(train['LoanAmount'].median())
train.loc[:, 'Loan_Amount_Term'] = train.loc[:, 'Loan_Amount_Term'].fillna(train['Loan_Amount_Term'].median())
train.loc[:, 'Credit_History'] = train.loc[:, 'Credit_History'].fillna(1.0)

In [108]:
test.loc[:, 'Married'] = test.loc[:, 'Married'].fillna(0)
test.loc[:, 'Gender'] = test.loc[:, 'Gender'].fillna(0)
test.loc[:, 'Dependents'] = test.loc[:, 'Dependents'].fillna(0)
test.loc[:, 'Self_Employed'] = test.loc[:, 'Self_Employed'].fillna(0)
test.loc[:, 'LoanAmount'] = test.loc[:, 'LoanAmount'].fillna(train['LoanAmount'].median())
test.loc[:, 'Loan_Amount_Term'] = test.loc[:, 'Loan_Amount_Term'].fillna(train['Loan_Amount_Term'].median())
test.loc[:, 'Credit_History'] = test.loc[:, 'Credit_History'].fillna(1.0)

test = test.replace(replace_dict)

In [109]:
train.shape,test.shape

((614, 20), (367, 19))

In [110]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,loan_null,married_null,dependents_null,self_employed_null,Loan_Amount_null,term_null,History_null
0,LP001002,1.0,0.0,0,1,0.0,5849,0.0,128.0,360.0,1.0,2,1,False,False,False,False,True,False,False
1,LP001003,1.0,1.0,1,1,0.0,4583,1508.0,128.0,360.0,1.0,0,0,False,False,False,False,False,False,False
2,LP001005,1.0,1.0,0,1,1.0,3000,0.0,66.0,360.0,1.0,2,1,False,False,False,False,False,False,False
3,LP001006,1.0,1.0,0,0,0.0,2583,2358.0,120.0,360.0,1.0,2,1,False,False,False,False,False,False,False
4,LP001008,1.0,0.0,0,1,0.0,6000,0.0,141.0,360.0,1.0,2,1,False,False,False,False,False,False,False


In [112]:
train['married_null']=train['married_null'].astype(int)
train['loan_null']=train['married_null'].astype(int)
train['dependents_null']=train['married_null'].astype(int)
train['self_employed_null']=train['married_null'].astype(int)
train['Loan_Amount_null']=train['married_null'].astype(int)
train['term_null']=train['married_null'].astype(int)
train['History_null']=train['married_null'].astype(int)

In [114]:
test['married_null']=test['married_null'].astype('int64')
test['loan_null']=test['married_null'].astype('int64')
test['dependents_null']=test['married_null'].astype('int64')
test['self_employed_null']=test['married_null'].astype('int64')
test['Loan_Amount_null']=test['married_null'].astype('int64')
test['term_null']=test['married_null'].astype('int64')
test['History_null']=test['married_null'].astype('int64')

In [115]:
train.drop(['Loan_ID'],axis=1,inplace=True)

In [116]:
train=train.astype('int64')

In [117]:
train.columns

Index(['Gender', 'Married', 'Dependents', 'Education', 'Self_Employed',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount',
       'Loan_Amount_Term', 'Credit_History', 'Property_Area', 'Loan_Status',
       'loan_null', 'married_null', 'dependents_null', 'self_employed_null',
       'Loan_Amount_null', 'term_null', 'History_null'],
      dtype='object')

In [118]:
cols=['Gender', 'loan_null', 'Married','married_null', 'Dependents','dependents_null', 'Education', 'Self_Employed','self_employed_null',
       'ApplicantIncome', 'CoapplicantIncome', 'LoanAmount','Loan_Amount_null',
       'Loan_Amount_Term','term_null',  'Credit_History', 'History_null', 'Property_Area', 'Loan_Status']

In [119]:
train2=pd.DataFrame(train[cols])

In [120]:
train2.head()

,Gender,loan_null,Married,married_null,Dependents,dependents_null,Education,Self_Employed,self_employed_null,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_null,Loan_Amount_Term,term_null,Credit_History,History_null,Property_Area,Loan_Status
0,1,0,0,0,0,0,1,0,0,5849,0,128,0,360,0,1,0,2,1
1,1,0,1,0,1,0,1,0,0,4583,1508,128,0,360,0,1,0,0,0
2,1,0,1,0,0,0,1,1,0,3000,0,66,0,360,0,1,0,2,1
3,1,0,1,0,0,0,0,0,0,2583,2358,120,0,360,0,1,0,2,1
4,1,0,0,0,0,0,1,0,0,6000,0,141,0,360,0,1,0,2,1


In [75]:
train2['Loan_Amount_null'].nunique()

2

In [113]:
train.head()

,Loan_ID,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,loan_null,married_null,dependents_null,self_employed_null,Loan_Amount_null,term_null,History_null
0,LP001002,1.0,0.0,0,1,0.0,5849,0.0,128.0,360.0,1.0,2,1,0,0,0,0,0,0,0
1,LP001003,1.0,1.0,1,1,0.0,4583,1508.0,128.0,360.0,1.0,0,0,0,0,0,0,0,0,0
2,LP001005,1.0,1.0,0,1,1.0,3000,0.0,66.0,360.0,1.0,2,1,0,0,0,0,0,0,0
3,LP001006,1.0,1.0,0,0,0.0,2583,2358.0,120.0,360.0,1.0,2,1,0,0,0,0,0,0,0
4,LP001008,1.0,0.0,0,1,0.0,6000,0.0,141.0,360.0,1.0,2,1,0,0,0,0,0,0,0


In [124]:
X_train = train.drop("Loan_Status", axis=1)
#X_train = X_train.drop("Loan_ID", axis=1)
Y_train = train["Loan_Status"].values
X_test = test.drop("Loan_ID", axis=1)
from sklearn.preprocessing import MinMaxScaler
min_max_scaler=MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(X_train)
X_train = pd.DataFrame(np_scaled)
#min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(X_test)
X_test = pd.DataFrame(np_scaled)

In [125]:
X_train

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17
0,1.0,0.0,0.00,1.0,0.0,0.070489,0.000000,0.172214,0.743590,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,1.0,0.25,1.0,0.0,0.054830,0.036192,0.172214,0.743590,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,1.0,0.00,1.0,1.0,0.035250,0.000000,0.082489,0.743590,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,1.0,0.00,0.0,0.0,0.030093,0.056592,0.160637,0.743590,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1.0,0.0,0.00,1.0,0.0,0.072356,0.000000,0.191027,0.743590,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,0.0,0.0,0.00,1.0,0.0,0.034014,0.000000,0.089725,0.743590,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
610,1.0,1.0,1.00,1.0,0.0,0.048930,0.000000,0.044863,0.358974,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
611,1.0,1.0,0.25,1.0,0.0,0.097984,0.005760,0.353111,0.743590,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
612,1.0,1.0,0.50,1.0,0.0,0.091936,0.000000,0.257598,0.743590,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [126]:
linreg = LinearRegression()
linreg.fit(X_train, Y_train)
Y_pred = linreg.predict(X_test)
acc_log = round(linreg.score(X_train, Y_train) * 100, 2)
acc_log

31.09

In [129]:
svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

80.94

In [130]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_pred = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

85.18

In [131]:
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
gaussian = GaussianNB()
gaussian.fit(X_train, Y_train)
Y_pred = gaussian.predict(X_test)
acc_gaussian = round(gaussian.score(X_train, Y_train) * 100, 2)
acc_gaussian

31.76

In [132]:
perceptron = Perceptron()
perceptron.fit(X_train, Y_train)
Y_pred = perceptron.predict(X_test)
acc_perceptron = round(perceptron.score(X_train, Y_train) * 100, 2)
acc_perceptron

80.46

In [135]:

linear_svc = LinearSVC()
linear_svc.fit(X_train, Y_train)
Y_pred = linear_svc.predict(X_test)
acc_linear_svc = round(linear_svc.score(X_train, Y_train) * 100, 2)
acc_linear_svc

81.27

In [136]:
sgd = SGDClassifier()
sgd.fit(X_train, Y_train)
Y_pred = sgd.predict(X_test)
acc_sgd = round(sgd.score(X_train, Y_train) * 100, 2)
acc_sgd

81.43

In [169]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_pred = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

100.0

In [181]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_pred = random_forest.predict(X_test)
random_forest.score(X_train, Y_train)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

100.0

In [180]:
extra=ExtraTreesClassifier(random_state=42)
extra.fit(X_train, Y_train)
extra.score(X_train, Y_train)
acc_extra = round(extra.score(X_train, Y_train) * 100, 2)
acc_extra

100.0

In [140]:
models = pd.DataFrame({
        'Model': ['Support Vector Machines', 'KNN', 'Linear Regression', 'Random Forest', 'Naive Bayes', 'Perceptron',
                  'Stochastic Gradient Descent', 'Linear SVC', 'Decision Tree','Extra Trees'],
        'Score': [acc_svc, acc_knn, acc_log, 
              acc_random_forest, acc_gaussian, acc_perceptron, 
              acc_sgd, acc_linear_svc, acc_decision_tree,acc_extra]
    })
models.sort_values(by='Score', ascending=False)

,Model,Score
3,Random Forest,100.00
8,Decision Tree,100.00
9,Extra Trees,100.00
1,KNN,85.18
6,Stochastic Gradient Descent,81.43
7,Linear SVC,81.27
0,Support Vector Machines,80.94
5,Perceptron,80.46
4,Naive Bayes,31.76
2,Linear Regression,31.09


In [170]:
Y_pred = Y_pred.tolist()
for i in range(len(Y_pred)):
    if Y_pred[i] == 1:
        Y_pred[i] = 'Y'
    else:
        Y_pred[i] = 'N'
y_pred = np.array(Y_pred)

In [171]:
submission = pd.DataFrame({
        "Loan_ID": test["Loan_ID"],
        "Loan_Status": Y_pred
    })
submission.to_csv('./submission.csv', index=False)

In [183]:
Y_pred_proba = random_forest.predict_proba(X_test)[:,1]
extra_proba=extra.predict_proba(X_test)[:,1]
dt_proba=decision_tree.predict_proba(X_test)[:,1]
#sgd_proba=sgd.predict_proba(X_test)[:,1]
knn_proba=knn.predict_proba(X_test)[:,1]

In [225]:
w1=0.3
w2=0.5
w4=0.2

In [254]:
final_proba=(Y_pred_proba*w1)+(extra_proba*w2)+(knn_proba*w4)

In [227]:
final_proba[0:5]

array([0.932     , 0.66733333, 0.861     , 0.80533333, 0.51866667])

In [228]:
Y_pred_proba[0:5]

array([0.84, 0.63, 0.82, 0.79, 0.59])

In [229]:
extra_proba[0:5]

array([0.96, 0.69, 0.83, 0.87, 0.55])

In [230]:
knn_proba[0:5]

array([1.        , 0.66666667, 1.        , 0.66666667, 0.33333333])

In [255]:
for i in range(len(final_proba)):
    if final_proba[i]>0.3:
        final_proba[i]=1
    else:
        final_proba[i]=0

In [249]:
final_proba[0:5]

array([1., 1., 1., 1., 0.])

In [250]:
Y_pred[0:5]

array([1, 1, 1, 1, 1], dtype=int64)

In [256]:
final_proba = final_proba.tolist()
for i in range(len(final_proba)):
    if final_proba[i] == 1:
        final_proba[i] = 'Y'
    else:
        final_proba[i] = 'N'
final = np.array(final_proba)

In [257]:
final[0:5]

array(['Y', 'Y', 'Y', 'Y', 'Y'], dtype='<U1')

In [258]:
submission = pd.DataFrame({
        "Loan_ID": test["Loan_ID"],
        "Loan_Status": final
    })
submission.to_csv('./submission2.csv', index=False)